In [121]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from bayes_opt import BayesianOptimization
import category_encoders as ce

In [155]:
train = pd.read_csv(r'train.csv', index_col=0)
test = pd.read_csv(r'test.csv', index_col=0)

In [141]:
#metrica
def MAE(actual, pred):
    return (np.mean(np.absolute(actual - pred)))

## Preprocesamiento

In [156]:
train = train.fillna(0)
test = test.fillna(0)

In [157]:
train = train.loc[:,['metroscubiertos', 'banos', 'metrostotales', 'garages', 'habitaciones', 'gimnasio', 'usosmultiples', 'piscina', 'idzona', 'precio','fecha']]
test = test.loc[:,['metroscubiertos', 'banos', 'metrostotales', 'garages', 'habitaciones', 'gimnasio', 'usosmultiples', 'piscina', 'idzona','fecha']]

In [158]:
#aplico label encoder para normalizar las claves
encoder = LabelEncoder()
train['idzona'] = encoder.fit_transform(train['idzona']).astype(np.int64)
test['idzona'] = encoder.fit_transform(train['idzona']).astype(np.int64)

In [160]:
encoder = ce.HashingEncoder(cols=['idzona'], n_components=200)
x_train = train.drop('precio', axis = 1)
y_train = train['precio']

x_test = test.drop('precio', axis = 1)
y_test = test['precio']

In [161]:
ids_zonas_train = encoder.fit_transform(x_train, y_train)
ids_zonas_test = encoder.fit_transform(x_test, y_test)

In [162]:
def zonas(self):
    lista = []
    for i in range (0, 200):
        lista.append('col_' + str(i))
        
    return lista

In [163]:
ids_zonas = ids_zonas[zonas]

In [164]:
ids_zonas = ids_zonas.reset_index().rename(columns={'index':'id'})
train = train.reset_index()

In [167]:
train = pd.merge(train, ids_zonas, on='id', how='inner')
#fin encoding

In [168]:
train


,metroscubiertos,banos,metrostotales,garages,habitaciones,gimnasio,usosmultiples,piscina,idzona,precio,...,col_190,col_191,col_192,col_193,col_194,col_195,col_196,col_197,col_198,col_199
id,,,,,,,,,,,,,,,,,,,,,
53461,268.0,2.0,180.0,2.0,3.0,0.0,0.0,0.0,3619,3600000.0,...,0,0,0,0,0,0,0,0,0,0
209067,63.0,1.0,67.0,1.0,2.0,0.0,0.0,0.0,7286,650000.0,...,0,0,0,0,0,0,0,0,0,0
185997,95.0,1.0,95.0,1.0,2.0,0.0,0.0,0.0,5992,1150000.0,...,0,0,0,0,0,0,0,0,0,0
126147,75.0,1.0,90.0,1.0,2.0,0.0,0.0,0.0,3055,1100000.0,...,0,0,0,0,0,0,0,0,0,0
139233,140.0,2.0,160.0,1.0,3.0,0.0,0.0,0.0,12292,1150000.0,...,0,0,0,0,0,0,0,0,0,0
5013,293.0,3.0,293.0,2.0,4.0,0.0,0.0,0.0,18557,4200000.0,...,0,0,0,0,0,0,0,0,0,0
44962,58.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,1833,310000.0,...,0,0,0,0,0,0,0,0,0,0
134537,250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9113,6200000.0,...,0,0,0,0,0,0,0,0,0,0
51180,100.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,14178,488000.0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
#Agregar feature con año
#Buscar feature hashing, probar con 400 columnas o por ahi (category_encoders biblioteca)
train["fecha"] = pd.to_datetime(train["fecha"])
train["año"] = train["fecha"].dt.year.astype(int)

test["fecha"] = pd.to_datetime(test["fecha"])
test["año"] = test["fecha"].dt.year.astype(int)

In [47]:
train_2012 = train[lambda df: df["año"] == 2012].drop(columns=["fecha","año"])
train_2013 = train[lambda df: df["año"] == 2013].drop(columns=["fecha","año"])
train_2014 = train[lambda df: df["año"] == 2014].drop(columns=["fecha","año"])
train_2015 = train[lambda df: df["año"] == 2015].drop(columns=["fecha","año"])
train_2016 = train[lambda df: df["año"] == 2016].drop(columns=["fecha","año"])

test_2012 = test[lambda df: df["año"] == 2012].drop(columns=["fecha","año"])
test_2013 = test[lambda df: df["año"] == 2013].drop(columns=["fecha","año"])
test_2014 = test[lambda df: df["año"] == 2014].drop(columns=["fecha","año"])
test_2015 = test[lambda df: df["año"] == 2015].drop(columns=["fecha","año"])
test_2016 = test[lambda df: df["año"] == 2016].drop(columns=["fecha","año"])

## Feature Engineering

In [7]:
#train['cant_amenities'] = train['usosmultiples'] + train['piscina'] + train['gimnasio']
#test['cant_amenities'] = test['usosmultiples'] + test['piscina'] + test['gimnasio']

## Bayesian Opt

In [91]:
def train_set(train,test):
    X = train.drop(['precio'], axis=1)
    y = train['precio']
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=1)
    print("Train shapes: X = " + str(X_train.shape) + " y = " + str(y_train.shape))
    print("Validation shapes: X = " + str(X_val.shape) +  " y = " + str(y_val.shape))
    print("Test shape: " + str(test.shape))
    return X, y, X_val, y_val

In [102]:
x_2012, y_2012, x_2012_val, y_2012_val = train_set(train_2012,test_2012)
x_2013, y_2013, x_2013_val, y_2013_val = train_set(train_2013,test_2013)
x_2014, y_2014, x_2014_val, y_2014_val = train_set(train_2014,test_2014)
x_2015, y_2015, x_2015_val, y_2015_val = train_set(train_2015,test_2015)
x_2016, y_2016, x_2016_val, y_2016_val = train_set(train_2016,test_2016)

Train shapes: X = (14120, 9) y = (14120,)
Validation shapes: X = (9414, 9) y = (9414,)
Test shape: (5899, 9)
Train shapes: X = (18231, 9) y = (18231,)
Validation shapes: X = (12155, 9) y = (12155,)
Test shape: (7592, 9)
Train shapes: X = (24343, 9) y = (24343,)
Validation shapes: X = (16229, 9) y = (16229,)
Test shape: (10018, 9)
Train shapes: X = (30882, 9) y = (30882,)
Validation shapes: X = (20588, 9) y = (20588,)
Test shape: (13017, 9)
Train shapes: X = (56422, 9) y = (56422,)
Validation shapes: X = (37616, 9) y = (37616,)
Test shape: (23474, 9)


In [69]:
def xgb_evaluate(max_depth, gamma, colsample_bytree,seed,min_child_weight,n_estimators,
                  reg_alpha,reg_lambda,subsample,learning_rate):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': subsample,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              "seed": int(seed),
              "min_child_weight": min_child_weight,
              "n_estimators": n_estimators,
              "reg_alpha": reg_alpha,
              "reg_lambda": reg_lambda,
              "learning_rate": learning_rate
             }
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=1000, nfold=5)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [103]:
def optimize(sett):
    dtrain = sett
    xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9),
                                             "seed": (10,50),
                                             "min_child_weight": (0.4,1.5),
                                             "n_estimators":(100,10000),                                                                    
                                             "reg_alpha":(0.2,0.75),
                                             "reg_lambda": (0.2,0.8),
                                             "subsample" : (0.3, 0.8),
                                             "learning_rate": (0.01,0.07),
                                            })
    # Se probo con 3 puntos iniciales y 10 iteraciones, deberiamos probar con mas en ambas
    # La prox iteracion en casa probar 100 y 10000
    xgb_bo.maximize(init_points=3, n_iter=10, acq='ei')
    params = xgb_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params["seed"] = int(params["seed"])
    params["n_estimators"] = int(params["n_estimators"])
    return params

In [70]:
import xgboost as xgb

dtrain_2012 = xgb.DMatrix(x_2012, label=y_2012)
dtrain_2013 = xgb.DMatrix(x_2013, label=y_2013)
dtrain_2014 = xgb.DMatrix(x_2014, label=y_2014)
dtrain_2015 = xgb.DMatrix(x_2015, label=y_2015)
dtrain_2016 = xgb.DMatrix(x_2016, label=y_2016)

In [107]:
#params_2012 = optimize(dtrain_2012)
params_2013 = optimize(dtrain_2013)
params_2014 = optimize(dtrain_2014)
params_2015 = optimize(dtrain_2015)
params_2016 = optimize(dtrain_2016)

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... |   seed    | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -1.021e+0 |  0.4891   |  0.7126   |  0.04916  |  4.89     |  0.9986   |  1.49e+03 |  0.2792   |  0.238    |  10.01    |  0.3896   |
|  2        | -9.987e+0 |  0.8964   |  0.01115  |  0.03592  |  6.956    |  0.9892   |  4.052e+0 |  0.6214   |  0.4992   |  42.17    |  0.7276   |
|  3        | -1.071e+0 |  0.3214   |  0.8952   |  0.01225  |  5.695    |  1.214    |  5.095e+0 |  0.4706   |  0.7673   |  12.44    |  0.3637   |
|  4        | -1.013e+0 |  0.7509   |  0.7154   |  0.04321  |  6.419    |  1.436    |  2.072e+0 |  0.3818   |  0.2102   |  24.07    |  0.4328   |
|  5        | -1.034e+0 |  0.5874   |  0.8906   |  0.04839  |  3.059    |  1.042    |  3.395e+0 |  0.6622   |  0.7887   |  1

|  4        | -1.01e+06 |  0.8856   |  0.3283   |  0.04036  |  6.157    |  1.071    |  4.378e+0 |  0.5359   |  0.2723   |  24.44    |  0.4348   |
|  5        | -1.028e+0 |  0.688    |  0.7806   |  0.0388   |  3.583    |  0.9871   |  4.614e+0 |  0.5417   |  0.6919   |  36.2     |  0.7874   |
|  6        | -9.95e+05 |  0.6615   |  0.8088   |  0.03825  |  6.668    |  0.6981   |  8.924e+0 |  0.2742   |  0.3791   |  47.27    |  0.6384   |
|  7        | -1.027e+0 |  0.346    |  0.7223   |  0.03097  |  4.824    |  0.6788   |  394.8    |  0.4154   |  0.2836   |  15.09    |  0.6568   |
|  8        | -1.057e+0 |  0.5447   |  0.2508   |  0.02004  |  3.916    |  1.086    |  291.3    |  0.2282   |  0.7639   |  21.57    |  0.6162   |
|  9        | -9.986e+0 |  0.5517   |  0.9334   |  0.01731  |  6.842    |  1.122    |  4.448e+0 |  0.6104   |  0.7209   |  29.94    |  0.756    |
|  10       | -1.004e+0 |  0.3589   |  0.1233   |  0.05579  |  5.339    |  0.6474   |  5.212e+0 |  0.2408   |  0.7078   |  1

## XGBoost

In [109]:
def xgboost(x,y,x_val,y_val, test, params):
    best_xgb_model = XGBRegressor(colsample_bytree=params["colsample_bytree"],
                 gamma=params["gamma"],                 
                 learning_rate=params["learning_rate"],
                 max_depth=params["max_depth"],
                 min_child_weight=params["min_child_weight"],
                 n_estimators=params["n_estimators"],                                                                    
                 reg_alpha=params["reg_alpha"],
                 reg_lambda=params["reg_lambda"],
                 subsample=params["subsample"],
                 seed=params["seed"])
    best_xgb_model.fit(x,y)
    XGB_pred_val = best_xgb_model.predict(x_val)
    XGB_mae = MAE(y_val, XGB_pred_val)
    print("MAE XGB: " + str(XGB_mae))
    XGB_pred_test = best_xgb_model.predict(test)
    res = pd.DataFrame(XGB_pred_test, index=test.index, columns=['precio'])
    res = res.rename(columns={'precio':'target'})
    
    return res, XGB_pred_val

In [110]:
res_2012, XGB_2012_pred_val = xgboost(x_2012, y_2012, x_2012_val, y_2012_val, test_2012, params_2012)
res_2013, XGB_2013_pred_val = xgboost(x_2013, y_2013, x_2013_val, y_2013_val, test_2013, params_2013)
res_2014, XGB_2014_pred_val = xgboost(x_2014, y_2014, x_2014_val, y_2014_val, test_2014, params_2014)
res_2015, XGB_2015_pred_val = xgboost(x_2015, y_2015, x_2015_val, y_2015_val, test_2015, params_2015)
res_2016, XGB_2016_pred_val = xgboost(x_2016, y_2016, x_2016_val, y_2016_val, test_2016, params_2016)


[01:13:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 252339.44328080007
[01:15:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 339416.4994137857
[01:20:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 322106.2475693612
[01:26:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE XGB: 462973.6636366299


In [120]:
#ensamblo todo y comparo
#XGB_mae = MAE(y_val_ensamblado, XGB_pred_val_ensamblado)
#print("MAE XGB: " + str(XGB_mae))

res = res_2012.append(res_2013).append(res_2014).append(res_2015).append(res_2016)
res.to_csv("workshop-submission-XGB_2.csv", header=True)

In [98]:
import matplotlib.pyplot as plt
fscores = pd.DataFrame({'X': list(best_xgb_model.get_fscore().keys()), 'Y': list(best_xgb_model.get_fscore().values())})
fscores.sort_values(by='Y').plot.bar(x='X')

AttributeError: 'XGBRegressor' object has no attribute 'get_fscore'